Here's a demonstration of the Modifiable Areal Unit Problem using randomly generated data.

###Imports

* Geopandas - pandas, but extended to use Shapely geometric objects
* Numpy - random, correlation coefficient
* Shapely - Geometric objects

In [ ]:
import geopandas as gpd
from numpy import random,corrcoef,arange
from shapely.geometry import Point,Polygon,LineString,MultiLineString

###Generate Data



In [ ]:
def build_correlated_gdf(stddev = 40,size = 1000, point_ct = 1000):

    x = random.randint(1, 1000)
    y = random.randint(1, 1000)
    point0 = Point(x, y)
    r1 = random.randint(1,1000)
    gdf = gpd.GeoDataFrame({'geometry':[point0],
                            'value':[r1],
                            'value2':[r1]}
                           ,index=[0])

    for i in range(point_ct):
        
        # Create a new point in the space, randomly placed
        x = random.randint(1, 1000)
        y = random.randint(1, 1000)
        newPoint = Point(x,y)

        # take the ten nearest points to this new point. take their average
        temp_gdf = gdf.copy()
        temp_gdf['dist'] = temp_gdf['geometry'].apply(lambda p: p.distance(newPoint))
        temp_gdf.sort_values('dist',inplace=True)
        vm = temp_gdf.head(10)['value'].mean()

        # use this value as the center of a normal distribution, and pick a value randomly on that distribution
        # second value takes the normal dist of diff
        valdiff = random.normal(0, stddev)
        valdiff2 = random.normal(valdiff,stddev)

        # if the value goes outside our bounds, just bounce it back inside
        if ((vm + valdiff) > size) | ((vm + valdiff) < 1):
            valdiff = -valdiff
        if ((vm + valdiff2) > size) | ((vm + valdiff2) < 1):
            valdiff2 = -valdiff2

        # add the point and the values to the end of the the ongoing geodataframe
        gdf = gdf.append(gpd.GeoDataFrame({'geometry': [newPoint],
                                'value': [vm+valdiff],
                                'value2': [vm+valdiff2]}
                                , index=[len(gdf)]))

    return gdf

def build_random_gdf(size = 1000, point_ct = 1000):

    vals = [random.randint(1,1000) for i in range(point_ct)]
    vals2 = [random.randint(1,1000) for i in range(point_ct)]
    points = [Point(random.randint(1,size),random.randint(1,size)) for i in range(point_ct)]

    return gpd.GeoDataFrame({'value':vals,'value2':vals2,'geometry':points})

In [ ]:

def unit_correlation(gdf,cell_size = 100.0):
    v1 = []
    v2 = []
    for i in arange(0,1000,cell_size):
        for j in arange(0,1000,cell_size):
            this_section = gdf.cx[i:(i+cell_size),j:(j+cell_size)]
            if len(this_section) == 0:
                continue
            else:
                v1.append(this_section['value'].mean())
                v2.append(this_section['value2'].mean())

    return corrcoef(v1,v2)[1,0]